# Implement Efficient weighted probabilistic frequent itemset mining in uncertain databases

In [17]:
import numpy as np
import random
import time


# 1. Preliminary

In [18]:
def get_probability_in_transaction(X, T):
    """get the probabilistic of itemset in transaction T

    Args:
        X (list<string>): itemset X
        T (dict<string, double>): transaction T

    Returns:
        prob: the probabilistic of itemset in transaction T
    """
    prob = 1.0
    for i in X:
        prob *= T.get(i, 0.0)
    return prob

def get_weighted_itemset(X, W):
    """get the weight of itemset in weighted table

    Args:
        X (list<string>): itemset X
        W (dict<string, double>): weighted table W

    Returns:
        weight of itemset: the weight of itemset in weighted table
    """
    weighted = 0

    for i in X:
        weighted += W.get(i, 0.0)

    return weighted / len(X)

def compute_support(X, D):
    """compute the support of itemset X

    Args:
        X (list<string>): itemset X
        D (list<dict<string, double>>): Uncertain database

    Returns:
        support: the support of itemset X
    """
    support = 0

    for Ti in D:
        probX= get_probability_in_transaction(X, Ti)
        
        if probX > 0:
            support += 1

    return support

def compute_expected_support(X, D):
    """compute the expected support of itemset X

    Args:
        X (list<string>): itemset X
        D (list<dict<string, double>>): Uncertain database

    Returns:
        expected_support: the expected support of itemset X
    """
    expected_support = 0

    for Ti in D:
        expected_support += get_probability_in_transaction(X, Ti)

    return expected_support

def get_min_weighted_itemset(X, W):
    min_weighted = 1

    for i in X:
        if W.get(i, 0.0) < min_weighted :
            min_weighted = W.get(i, 0.0)

    return min_weighted

In [19]:
def compute_prWF(X, D, W, min_conf):
    """compute the probabilistic support of itemset X

    Args:
        X (list<string>): itemset X
        D (list<dict<string, double>>): Uncertain database
        W (dict<string, double): Weighted table
        min_conf (double): min confidence

    Returns:
        probabilistic support vector: the probabilistic support vector of itemset X
    """
    convolution_vector = compute_support_probabilistic_vector(X, D)
    weighted = get_weighted_itemset(X, W)
    prWF = 0
    
    for i in range(len(D), 0, -1):
        prWF += convolution_vector[i]*weighted

        if prWF > min_conf:
            return i

    return -1

def compute_support_probabilistic_vector(X, D):
    """calculate support probabilistic vector of itemset X in D 

    Args:
        X (list<string>): itemset X
        D (list<dict<string, double>>): Uncertain database

    Returns:
        support_probabilistic_vector: support probabilistic vector of itemset X in D 
    """
    n = len(D)
    
    # Base case
    if n == 1:
        prob = get_probability_in_transaction(X, D[0])
        f_X = np.zeros(2)
        f_X[0] = 1 - prob
        f_X[1] = prob
        return f_X

    # Recursive case: Partition PDB
    mid = n // 2
    D1 = D[:mid]
    D2 = D[mid:]

    # Recursively call DC on partitions
    f_X1 = compute_support_probabilistic_vector(X, D1)
    f_X2 = compute_support_probabilistic_vector(X, D2)

    # Convolution 2 array
    # Adjust length of f_X1 and f_X2 for convolution
    len_total = len(f_X1) + len(f_X2) - 1
    f_X1 = np.pad(f_X1, (0, len_total - len(f_X1)), mode='constant')
    f_X2 = np.pad(f_X2, (0, len_total - len(f_X2)), mode='constant')

    # Convolution using FFT
    f_X = np.fft.ifft(np.fft.fft(f_X1) * np.fft.fft(f_X2)).real
    f_X = f_X[:n+1]  # Take only the first n+1 elements
    f_X = np.round(f_X, decimals=10)  # Fix numerical inaccuracies

    return f_X

# 2. Algorithms

In [20]:
def algorithms(D, W, min_sup, min_conf):
    
    I = discover_single_items(D)
    wPFI_k = Scan_Find_Size_k_wPMFI(I, D, W, min_sup, min_conf)
    
    wPFI = [wPFI_k]
        
    while len(wPFI_k) > 0:
        Ck = wPFIAprioriGen(wPFI_k, I, W, min_conf)
        # print("Ck", Ck)
        wPFI_k = Scan_Find_Size_k_wPMFI(Ck, D, W, min_sup, min_conf)
        
        if len(wPFI_k) > 0:
            wPFI.append(wPFI_k)
                    
    return find_all_wpmfi(wPFI)

def discover_single_items(D):
    # Implement discovery of single items using Chernoff-Hoeffding bound
    single_items = set()

    for Ti in D:
        for item in Ti.keys():
            single_items.add(item)

    return single_items
    
def Scan_Find_Size_k_wPMFI(candidateK, D, W, min_sup, min_conf):
    result = set()
    
    for item in candidateK:
        prob_sup = compute_prWF(item, D, W, min_conf)
        # print(item , " - ", prob_sup)
        if prob_sup >= min_sup:
            result.add(frozenset(item))
    
    return result

def wPFIAprioriGen(wPFI_prev, I, W, min_conf):
    Ck = set()
    
    _I = set()
    
    for itemset in wPFI_prev:
        _I.update(set(itemset))
                
    for itemset in wPFI_prev:
        diffSet = _I - set(itemset)
                
        for Ii in diffSet:
            unionCandidate = set(itemset)
            unionCandidate.add(Ii)
                        
            if get_weighted_itemset(unionCandidate, W) >= min_conf:
                Ck.add(frozenset(unionCandidate))
        
        Im = get_min_weighted_itemset(itemset, W)
        
        diffSet = set()
        
        for item in I:
            diffSet.update(item)
            
        diffSet = diffSet - _I
        diffSet = diffSet - set(itemset)
        
        for Ii in diffSet:
            unionCandidate = set()
            unionCandidate.update(set(itemset))
            unionCandidate.update(set([Ii]))
            
            if get_min_weighted_itemset(unionCandidate, W) >= min_conf and get_weighted_itemset(Ii, W) < Im:
                Ck.add(frozenset(unionCandidate))
        
    return Ck
        
def find_all_wpmfi(wpfi):
    wpmfi = set()

    # find itemset X is wPMFI;
    # itemset X is wPMFI: not find itemset Y so that Y is wPFI and Y covered itemset X
    for i in range(len(wpfi) - 1):
        wpf_i_k = wpfi[i]
        wpf_i_k_plus_1 = wpfi[i + 1]

        # traversal 2 set size-k and size-(k+1)
        # if itemset in size-(k+1) didn't cover itemset size_k -> itemset size_k is wPMFI
        for itemset_wpf_i_k in wpf_i_k:
            is_contain_in_larger_itemset = False
            for itemset_wpf_i_k_plus_1 in wpf_i_k_plus_1:
                if itemset_wpf_i_k.issubset(itemset_wpf_i_k_plus_1):
                    is_contain_in_larger_itemset = True
                    break
            if not is_contain_in_larger_itemset:
                wpmfi.add(frozenset(itemset_wpf_i_k))

    # add last set in wPFI; because it always maximal
    last_index_in_wpfi = len(wpfi) - 1
    wpmfi.update(frozenset(itemset) for itemset in wpfi[last_index_in_wpfi])

    return wpmfi


# 3. Example

In [21]:
# Example 1
D = [
    {'A': 0.6, 'B': 0.7},
    {'A': 0.2, 'C': 0.3},
]

weighted = {
    'A': 1,
    'B': 1,
    'C': 1
}

min_sup = 1
wfpt = 0.1

# #Example 2
D = [
    {"1": 0.5, "2": 0.7, "4": 0.8, "5": 0.9},
    {"2": 0.6, "3": 0.8, "4": 0.6, "5": 0.8},
    {"3": 0.6, "4": 0.9, "5": 0.5},
    {"1": 0.6, "3": 0.7, "4": 0.8, "5": 0.8},
    {"1": 0.8, "2": 0.9, "3": 0.5, "4": 0.6, "5": 0.7},
    {"2": 0.6, "4": 0.9, "5": 0.8},
]

weighted = {
    "1": 0.3,
    "2": 0.9,
    "3": 0.5,
    "4": 0.6,
    "5": 0.9
}

min_sup = 2
wfpt = 0.2
# print(candidate_generate_expected_bound(D, min_sup, wfpt))
print(algorithms(D, weighted, min_sup, wfpt))

1  -  2
5  -  5
3  -  3
4  -  5
2  -  4
Ck {frozenset({'1', '5'}), frozenset({'2', '4'}), frozenset({'1', '2'}), frozenset({'2', '5'}), frozenset({'4', '3'}), frozenset({'1', '4'}), frozenset({'5', '3'}), frozenset({'1', '3'}), frozenset({'4', '5'}), frozenset({'2', '3'})}
frozenset({'1', '5'})  -  2
frozenset({'2', '4'})  -  3
frozenset({'1', '2'})  -  1
frozenset({'2', '5'})  -  3
frozenset({'4', '3'})  -  2
frozenset({'1', '4'})  -  1
frozenset({'5', '3'})  -  2
frozenset({'1', '3'})  -  1
frozenset({'4', '5'})  -  4
frozenset({'2', '3'})  -  1
Ck {frozenset({'1', '2', '4'}), frozenset({'1', '2', '5'}), frozenset({'2', '4', '5'}), frozenset({'2', '4', '3'}), frozenset({'1', '4', '5'}), frozenset({'1', '4', '3'}), frozenset({'5', '1', '3'}), frozenset({'5', '4', '3'}), frozenset({'2', '3', '5'})}
frozenset({'1', '2', '4'})  -  1
frozenset({'1', '2', '5'})  -  1
frozenset({'2', '4', '5'})  -  2
frozenset({'2', '4', '3'})  -  1
frozenset({'1', '4', '5'})  -  1
frozenset({'1', '4', '3'}

# 4. Real dataset

In [ ]:
def read_dataset(file_path, mean, variance):
    uncertain_database = []

    # Initialize normal distribution
    random.seed(12345)

    try:
        with open(file_path, 'r') as file:
            curr_id_transaction = 0
            count_num_line = 1

            for line in file:
                data_line_transaction = line.strip().split(" ")
                cur_transaction = {}

                for item in data_line_transaction:
                    data_item = item.split("-")
                    value = data_item[0]
                    prob = float(data_item[1])
                    cur_transaction[value] = prob

                uncertain_database.append(cur_transaction)
                curr_id_transaction += 1
                count_num_line += 1
    except IOError as e:
        print(e)

    print(uncertain_database)

    return uncertain_database

def generate_weighted_table(uncertain_database):
    weighted_table = {}
    distinct_itemset_database = discover_single_items(uncertain_database)

    random.seed(12345)

    for item in distinct_itemset_database:
        weighted = round(random.random(), 1)
        if weighted <= 0:
            weighted = 0.1
        else:
            if weighted >= 1:
                weighted = 0.9
        weighted_table[item] = weighted

    return weighted_table

## Dataset

In [ ]:
path = './data/accidents_10K.data'
path = './data/connect4_10K.data'
path = './data/T40I10D100K_10K.data'
path = './data/us_10K.data'
D = read_dataset(path, 0.78, 0.65)
W = generate_weighted_table(D)

for i in range(0, 5):
    print(D[i])

[{'1': 0.4, '2': 0.7, '3': 0.8, '4': 0.3, '5': 0.1, '6': 0.3, '7': 0.2, '8': 0.9, '9': 0.3, '10': 0.7, '11': 1.0, '12': 0.5, '13': 0.1, '14': 0.9, '15': 0.7, '16': 0.4, '17': 0.2, '18': 0.5, '19': 0.2, '20': 0.3, '21': 0.5, '22': 0.4, '23': 0.4, '24': 0.7, '25': 0.3, '26': 0.0, '27': 0.2, '28': 0.6, '29': 0.1, '30': 0.5, '31': 0.6}, {'2': 0.8, '5': 0.5, '7': 0.7, '8': 0.1, '9': 0.9, '10': 1.0, '12': 0.5, '13': 0.8, '14': 0.9, '15': 0.6, '16': 0.2, '17': 0.4, '18': 0.1, '20': 0.2, '22': 0.1, '23': 0.9, '24': 0.9, '25': 0.1, '27': 0.7, '28': 0.9, '29': 1.0, '32': 0.9, '33': 0.8, '34': 0.6, '35': 0.9, '36': 1.0, '37': 0.6, '38': 0.4, '39': 0.8}, {'7': 0.6, '10': 0.7, '12': 0.9, '13': 0.9, '14': 0.1, '15': 0.1, '16': 0.3, '17': 0.1, '18': 0.5, '20': 0.3, '25': 0.8, '28': 0.6, '29': 0.1, '30': 0.9, '33': 0.2, '40': 0.5, '41': 0.5, '42': 0.3, '43': 0.5, '44': 0.7, '45': 0.1, '46': 0.4, '47': 0.4, '48': 0.7, '49': 0.5, '50': 0.4, '51': 0.9, '52': 0.8}, {'1': 0.3, '5': 0.6, '8': 0.6, '10': 0.6

## Min support and minconfidence

In [ ]:
min_sup = 0.1*10000
min_conf = 0.7

# 5. Run algorithm

In [ ]:
# Start the timer
start_time = time.time()

# Your code here
result = algorithms(D, W, min_sup, min_conf)
print(result)
print(len(result))

# End the timer
end_time = time.time()

# Calculate the runtime
runtime = end_time - start_time
print(f"Runtime: {runtime} seconds")


C:\Users\hung.tt\AppData\Local\Temp\ipykernel_392\2239993570.py:2: RuntimeWarning: invalid value encountered in sqrt
  return (2*min_sup - np.log(min_conf) - np.sqrt((np.log(min_conf))**2 - 8*min_sup*np.log(min_conf))) / 2


[['1', '22'], ['1', '41'], ['1', '25'], ['1', '66'], ['22', '41'], ['22', '25'], ['22', '38'], ['22', '66'], ['41', '25'], ['41', '38'], ['41', '30'], ['41', '66'], ['25', '38'], ['10', '25'], ['25', '66'], ['38', '66'], ['23', '66'], ['10', '66'], ['30', '66'], ['80'], ['5'], ['82']]
22
Runtime: 16.93607759475708 seconds
